In [1]:
library(ggplot2)
library(GenomicRanges)
library("stats")
library(scales)
library(dplyr)  # Carica il pacchetto dplyr
bnoExt=function(x){
return(basename(tools::file_path_sans_ext(x)))
}
fNumber=function(x){
sapply(x,FUN=function(x){
strsplit(x,"__")[[1]][2]
})
}
# Definisci colori e tipi di linee
color_mapping <- c(
  "AID_KO_DMSO" = "black",
  "AID_KO_Taze" = "black",
  "AID_KO_Vale" = "black",
  "Ligase4_KO_DMSO" = "blue",
  "Ligase4_KO_Taze" = "red",
  "Ligase4_KO_Vale" = "orange",
  "AID_WT_DMSO" = "purple",
  "AID_WT_Taze" = "darkgreen",
  "AID_WT_Vale" = "brown",
  "Idelalisib_DMSO" = "blue",
  "Idelalisib_Taze" = "red",
  "Idelalisib_Vale" = "orange"
)

linetype_mapping <- c(
  "AID_KO_DMSO" = "solid",
  "AID_KO_Taze" = "dashed",
  "AID_KO_Vale" = "dotted",
  "Ligase4_KO_DMSO" = "solid",
  "Ligase4_KO_Taze" = "solid",
  "Ligase4_KO_Vale" = "solid",
  "AID_WT_DMSO" = "solid",
  "AID_WT_Taze" = "solid",
  "AID_WT_Vale" = "solid",
  "Idelalisib_DMSO" = "solid",
  "Idelalisib_Taze" = "solid",
  "Idelalisib_Vale" = "solid"
)


Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    Filter, Find, Map, Position, Reduce, anyDuplicated, append,
    as.data.frame, basename, cbind, colnames, dirname, do.call,
    duplicated, eval, evalq, get, grep, grepl, intersect, is.unsorted,
    lapply, mapply, match, mget, order, paste, pmax, pmax.int, pmin,
    pmin.int, rank, rbind, rownames, sapply, setdiff, sort, table,
    tapply, union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:base’:

    I, expand.grid, unname


Loading required package: IRanges

Loading required package: GenomeInfoDb


Attaching package: ‘dplyr’


The following objects are masked from ‘package:GenomicRanges’:

    intersect, setdiff, union

In [27]:
# Carica i dati dei file BED
print("bonjovy")
tss_file <- "/sharedFolder/Data/mm9_site_TSS.bed"
tss_file=read.table(tss_file,header=FALSE,sep="\t")
colnames(tss_file)=c("chrom", "start", "end", "gene", "score", "strand")
tss_gr <- GRanges(seqnames = tss_file$chrom, IRanges(start = tss_file$start, end = tss_file$end))


all_data <- data.frame()
all_data2 <- data.frame()

for(junctions_file in list.files("/sharedFolder/Data/1_HTGTS/3_JunctionPaper/junction_pooled/")[-c(1,2,3)])
{
#junctions_file=list.files("/sharedFolder/Data/1_HTGTS/3_JunctionPaper/junction_pooled/")[1]
junctions_file=paste("/sharedFolder/Data/1_HTGTS/3_JunctionPaper/junction_pooled/",junctions_file,sep="")

junctions=read.table(junctions_file,header=FALSE,sep="\t")
colnames(junctions)=c("chrom", "start", "end", "junction", "score", "strand")
junctions_gr <- GRanges(seqnames = junctions$chrom, IRanges(start = junctions$start, end = junctions$end))

#REMOVE MYC and IGH LOCUS 
myc=GRanges(seqnames="chr15",ranges=IRanges(start=60562682,end=63076130))
toRemove=findOverlaps(myc,junctions_gr)@to
igh=GRanges(seqnames="chr12",ranges=IRanges(start=114419999,end=114680000))
toRemove=append(toRemove,findOverlaps(igh,junctions_gr)@to)
junctions_gr=junctions_gr[-toRemove,]
#############################


distance_GR=distanceToNearest(junctions_gr,tss_gr)
distance=start(junctions_gr[queryHits(distance_GR)])- start(tss_gr[subjectHits(distance_GR)])


rpm=log2(countOverlaps(junctions_gr[queryHits(distance_GR)],promoters(junctions_gr,upstream=1000,downstream=1000))+1)
    james=rpm
f=smooth.spline(distance,rpm,spar=0.6)
############################
distanze=distance
intervalli <- seq(floor(min(distanze)), ceiling(max(distanze)), by = 10000)
conteggio_intervallo <- cut(distanze, breaks = intervalli,labels=intervalli[-1])
conteggio_df <- data.frame(Intervallo = as.numeric(levels(conteggio_intervallo)),
                           Frequenza = table(conteggio_intervallo))
conteggio_df=conteggio_df[,-2]
#f2=smooth.spline(conteggio_df[,1],conteggio_df[,2],spar=0)    #SPLINE
loess_fit <- loess(conteggio_df[,2] ~ conteggio_df[,1])
loess_values <- predict(loess_fit)
f2=cbind(conteggio_df[,1],loess_values)
colnames(f2)=c("x","y")
f2=data.frame(f2)    
#############################
cycle_data <- data.frame(distance = f$x, rpm = f$y, cycle = fNumber(bnoExt(basename(junctions_file))))#
cycle_data2 <- data.frame(distance = f2$x, counts = f2$y, cycle = fNumber(bnoExt(basename(junctions_file))))#  SPLINE
#cycle_data2 <- data.frame(distance = conteggio_df[,1], counts = conteggio_df[,2], cycle = fNumber(bnoExt(basename(junctions_file))))# RAW
    
    
all_data <- rbind(all_data, cycle_data)
all_data2 <- rbind(all_data2, cycle_data2)

}
all_data=as.data.frame(all_data)

# Crea una colonna separata per gli aloni
highlighted_cycles <- c("AID_KO_Vale", "AID_KO_Taze")
aura_data <- subset(all_data, cycle %in% highlighted_cycles)
aura_data$aura_cycle <- aura_data$cycle

# Colori specifici per l’aura (chiari e distinguibili)
aura_colors <- c(
  "AID_KO_Vale_aura" = "grey",  # blu
  "AID_KO_Taze_aura" = "lightblue"   # #377EB8
)


# Unione tra mapping delle linee e mapping degli aloni
# Assicurati che aura_cycle sia un fattore (questo è il trucco!)
aura_data$aura_cycle <- paste0(aura_data$cycle, "_aura")

plot <- ggplot() +
  geom_line(
    data = aura_data,
    aes(x = distance, y = rpm, color = aura_cycle, group = aura_cycle),
    size = 2.5, alpha = 0.4, show.legend = FALSE
  ) +
  geom_line(
    data = all_data,
    aes(x = distance, y = rpm, color = cycle, linetype = cycle),
    size = 0.9
  ) +
scale_color_manual(values = c(color_mapping, aura_colors)) +
  scale_linetype_manual(values = linetype_mapping) +
  labs(x = "Distance from TSS", y = "Spline fitted junctions distribution",
       title = "Spline fitted junctions distribution vs TSS distance") +
  theme_minimal() +
  theme(legend.position = "top", panel.grid = element_blank()) +
  coord_cartesian(xlim = c(-10000, 10000))

ggsave("/sharedFolder/TSS_WT.pdf", plot, bg = "white")

[1] "bonjovy"


Warning message in .Seqinfo.mergexy(x, y):
“Each of the 2 combined objects has sequence levels not in the other:
  - in 'x': chrM
  - in 'y': chrY
  Make sure to always combine/compare objects based on the same reference
  genome (use suppressWarnings() to suppress this warning).”
Warning message in data.frame(distance = f$x, rpm = f$y, cycle = fNumber(bnoExt(basename(junctions_file)))):
“row names were found from a short variable and have been discarded”
Warning message in data.frame(distance = f2$x, counts = f2$y, cycle = fNumber(bnoExt(basename(junctions_file)))):
“row names were found from a short variable and have been discarded”
Warning message in data.frame(distance = f$x, rpm = f$y, cycle = fNumber(bnoExt(basename(junctions_file)))):
“row names were found from a short variable and have been discarded”
Warning message in data.frame(distance = f2$x, counts = f2$y, cycle = fNumber(bnoExt(basename(junctions_file)))):
“row names were found from a short variable and have been discard

In [23]:
#THREE COLORS 
# Carica i dati dei file BED
print("bonjovy")
tss_file <- "/sharedFolder/Data/mm9_site_TSS.bed"
tss_file=read.table(tss_file,header=FALSE,sep="\t")
colnames(tss_file)=c("chrom", "start", "end", "gene", "score", "strand")
tss_gr <- GRanges(seqnames = tss_file$chrom, IRanges(start = tss_file$start, end = tss_file$end))


all_data <- data.frame()
all_data2 <- data.frame()

for(junctions_file in list.files("/sharedFolder/Data/1_HTGTS/3_JunctionPaper/junction_pooled/")[-c(1,2,3)])
{
#junctions_file=list.files("/sharedFolder/Data/1_HTGTS/3_JunctionPaper/junction_pooled/")[1]
junctions_file=paste("/sharedFolder/Data/1_HTGTS/3_JunctionPaper/junction_pooled/",junctions_file,sep="")

junctions=read.table(junctions_file,header=FALSE,sep="\t")
colnames(junctions)=c("chrom", "start", "end", "junction", "score", "strand")
junctions_gr <- GRanges(seqnames = junctions$chrom, IRanges(start = junctions$start, end = junctions$end))

#REMOVE MYC and IGH LOCUS 
myc=GRanges(seqnames="chr15",ranges=IRanges(start=60562682,end=63076130))
toRemove=findOverlaps(myc,junctions_gr)@to
igh=GRanges(seqnames="chr12",ranges=IRanges(start=114419999,end=114680000))
toRemove=append(toRemove,findOverlaps(igh,junctions_gr)@to)
junctions_gr=junctions_gr[-toRemove,]
#############################


distance_GR=distanceToNearest(junctions_gr,tss_gr)
distance=start(junctions_gr[queryHits(distance_GR)])- start(tss_gr[subjectHits(distance_GR)])


rpm=log2(countOverlaps(junctions_gr[queryHits(distance_GR)],promoters(junctions_gr,upstream=1000,downstream=1000))+1)
    james=rpm
f=smooth.spline(distance,rpm,spar=0.6)
############################
distanze=distance
intervalli <- seq(floor(min(distanze)), ceiling(max(distanze)), by = 10000)
conteggio_intervallo <- cut(distanze, breaks = intervalli,labels=intervalli[-1])
conteggio_df <- data.frame(Intervallo = as.numeric(levels(conteggio_intervallo)),
                           Frequenza = table(conteggio_intervallo))
conteggio_df=conteggio_df[,-2]
#f2=smooth.spline(conteggio_df[,1],conteggio_df[,2],spar=0)    #SPLINE
loess_fit <- loess(conteggio_df[,2] ~ conteggio_df[,1])
loess_values <- predict(loess_fit)
f2=cbind(conteggio_df[,1],loess_values)
colnames(f2)=c("x","y")
f2=data.frame(f2)    
#############################
cycle_data <- data.frame(distance = f$x, rpm = f$y, cycle = fNumber(bnoExt(basename(junctions_file))))#
cycle_data2 <- data.frame(distance = f2$x, counts = f2$y, cycle = fNumber(bnoExt(basename(junctions_file))))#  SPLINE
#cycle_data2 <- data.frame(distance = conteggio_df[,1], counts = conteggio_df[,2], cycle = fNumber(bnoExt(basename(junctions_file))))# RAW
    
    
all_data <- rbind(all_data, cycle_data)
all_data2 <- rbind(all_data2, cycle_data2)

}
all_data=as.data.frame(all_data)

# Crea una colonna separata per gli aloni
highlighted_cycles <- c("AID_KO_Vale", "AID_KO_Taze")
aura_data <- subset(all_data, cycle %in% highlighted_cycles)
aura_data$aura_cycle <- aura_data$cycle

# Colori specifici per l’aura (chiari e distinguibili)
aura_colors <- c(
  "AID_KO_Vale_aura" = "grey",  # blu
  "AID_KO_Taze_aura" = "lightblue"   # #377EB8
)


# Unione tra mapping delle linee e mapping degli aloni
# Assicurati che aura_cycle sia un fattore (questo è il trucco!)
aura_data$aura_cycle <- paste0(aura_data$cycle, "_aura")

plot <- ggplot() +
  geom_line(
    data = aura_data,
    aes(x = distance, y = rpm, color = aura_cycle, group = aura_cycle),
    size = 2.5, alpha = 0.4, show.legend = FALSE
  ) +
  geom_line(
    data = all_data,
    aes(x = distance, y = rpm, color = cycle, linetype = cycle),
    size = 0.9
  ) +
scale_color_manual(values = c(color_mapping, aura_colors)) +
  scale_linetype_manual(values = linetype_mapping) +
  labs(x = "Distance from TSS", y = "Spline fitted junctions distribution",
       title = "Spline fitted junctions distribution vs TSS distance") +
  theme_minimal() +
  theme(legend.position = "top", panel.grid = element_blank()) +
  coord_cartesian(xlim = c(-10000, 10000))

ggsave("/sharedFolder/TSS_WT.pdf", plot, bg = "white")






# Definisci gli intervalli
#ggplot(all_data2, aes(x = distance, y = counts, color = cycle, linetype = cycle)) +
#geom_line(data = all_data2, aes(x = distance, y = counts)) + #
#scale_color_manual(values = color_mapping) +
#scale_linetype_manual(values = linetype_mapping) +
#labs(x = "Distance from TSS", y = "log10(Counts) 2kb", title = "log10(Counts) (2kb) vs TSS distance") +
#  theme_minimal() +
#  theme(legend.position = "top") +
#  coord_cartesian(xlim = c(-10000,10000))#

[1] "bonjovy"


Warning message in .Seqinfo.mergexy(x, y):
“Each of the 2 combined objects has sequence levels not in the other:
  - in 'x': chrM
  - in 'y': chrY
  Make sure to always combine/compare objects based on the same reference
  genome (use suppressWarnings() to suppress this warning).”
Warning message in data.frame(distance = f$x, rpm = f$y, cycle = fNumber(bnoExt(basename(junctions_file)))):
“row names were found from a short variable and have been discarded”
Warning message in data.frame(distance = f2$x, counts = f2$y, cycle = fNumber(bnoExt(basename(junctions_file)))):
“row names were found from a short variable and have been discarded”
Warning message in data.frame(distance = f$x, rpm = f$y, cycle = fNumber(bnoExt(basename(junctions_file)))):
“row names were found from a short variable and have been discarded”
Warning message in data.frame(distance = f2$x, counts = f2$y, cycle = fNumber(bnoExt(basename(junctions_file)))):
“row names were found from a short variable and have been discard

In [28]:
# Carica i dati dei file BED
print("bonjovy")
tss_file <- "/sharedFolder/Data/mm9_site_TSS.bed"
tss_file=read.table(tss_file,header=FALSE,sep="\t")
colnames(tss_file)=c("chrom", "start", "end", "gene", "score", "strand")
tss_gr <- GRanges(seqnames = tss_file$chrom, IRanges(start = tss_file$start, end = tss_file$end))


all_data <- data.frame()

for(junctions_file in list.files("/sharedFolder/Data/1_HTGTS/3_JunctionPaper/junction_pooled/")[c(1,2,3,7,8,9)])#,7,8,9)])
{
#junctions_file=list.files("/sharedFolder/Data/1_HTGTS/3_JunctionPaper/junction_pooled/")[1]
junctions_file=paste("/sharedFolder/Data/1_HTGTS/3_JunctionPaper/junction_pooled/",junctions_file,sep="")

junctions=read.table(junctions_file,header=FALSE,sep="\t")
colnames(junctions)=c("chrom", "start", "end", "junction", "score", "strand")
junctions_gr <- GRanges(seqnames = junctions$chrom, IRanges(start = junctions$start, end = junctions$end))

#REMOVE MYC and IGH LOCUS 
myc=GRanges(seqnames="chr15",ranges=IRanges(start=60562682,end=63076130))
toRemove=findOverlaps(myc,junctions_gr)@to
igh=GRanges(seqnames="chr12",ranges=IRanges(start=114419999,end=114680000))
toRemove=append(toRemove,findOverlaps(igh,junctions_gr)@to)
junctions_gr=junctions_gr[-toRemove,]
#############################


distance_GR=distanceToNearest(junctions_gr,tss_gr)
distance=start(junctions_gr[queryHits(distance_GR)])- start(tss_gr[subjectHits(distance_GR)])


rpm=log2(countOverlaps(promoters(junctions_gr[queryHits(distance_GR)],upstream=1000,downstream=1000),junctions_gr)+1)
    james=countOverlaps(junctions_gr[queryHits(distance_GR)],promoters(junctions_gr,upstream=1000,downstream=1000))
f=smooth.spline(distance,rpm,spar=0.6)#,spar=0.6)
############################


cycle_data <- data.frame(distance = f$x, rpm = f$y, cycle = fNumber(bnoExt(basename(junctions_file))))#
all_data <- rbind(all_data, cycle_data)

}
all_data=as.data.frame(all_data)
# Crea una colonna separata per gli aloni
highlighted_cycles <- c("AID_KO_Vale", "AID_KO_Taze")
aura_data <- subset(all_data, cycle %in% highlighted_cycles)
aura_data$aura_cycle <- aura_data$cycle

# Colori specifici per l’aura (chiari e distinguibili)
aura_colors <- c(
  "AID_KO_Vale_aura" = "grey",  # blu
  "AID_KO_Taze_aura" = "lightblue"   # #377EB8
)


# Unione tra mapping delle linee e mapping degli aloni
# Assicurati che aura_cycle sia un fattore (questo è il trucco!)
aura_data$aura_cycle <- paste0(aura_data$cycle, "_aura")

plot <- ggplot() +
  geom_line(
    data = aura_data,
    aes(x = distance, y = rpm, color = aura_cycle, group = aura_cycle),
    size = 2.5, alpha = 0.4, show.legend = FALSE
  ) +
  geom_line(
    data = all_data,
    aes(x = distance, y = rpm, color = cycle, linetype = cycle),
    size = 0.9
  ) +
scale_color_manual(values = c(color_mapping, aura_colors)) +
  scale_linetype_manual(values = linetype_mapping) +
  labs(x = "Distance from TSS", y = "Spline fitted junctions distribution",
       title = "Spline fitted junctions distribution vs TSS distance") +
  theme_minimal() +
  theme(legend.position = "top", panel.grid = element_blank()) +
  coord_cartesian(xlim = c(-10000, 10000))

ggsave("/sharedFolder/TSS_LIG.pdf", plot, bg = "white")


[1] "bonjovy"


Warning message in data.frame(distance = f$x, rpm = f$y, cycle = fNumber(bnoExt(basename(junctions_file)))):
“row names were found from a short variable and have been discarded”
Warning message in data.frame(distance = f$x, rpm = f$y, cycle = fNumber(bnoExt(basename(junctions_file)))):
“row names were found from a short variable and have been discarded”
Warning message in data.frame(distance = f$x, rpm = f$y, cycle = fNumber(bnoExt(basename(junctions_file)))):
“row names were found from a short variable and have been discarded”
Warning message in .Seqinfo.mergexy(x, y):
“Each of the 2 combined objects has sequence levels not in the other:
  - in 'x': chrM
  - in 'y': chrY
  Make sure to always combine/compare objects based on the same reference
  genome (use suppressWarnings() to suppress this warning).”
Warning message in data.frame(distance = f$x, rpm = f$y, cycle = fNumber(bnoExt(basename(junctions_file)))):
“row names were found from a short variable and have been discarded”
Warnin

In [22]:
print("bonjovy")
Groseq="/sharedFolder/Data/3_gro_seq/bed/"
condition=c("DMSO","Taze","Vale")
for(i in condition){
#i=condition[1]
j=list.files(Groseq)[grep(i,list.files(Groseq))]
jPos=j[grep("pos",j)]
jNeg=j[grep("neg",j)]
filePos=paste(Groseq,jPos,sep="/")
fileNeg=paste(Groseq,jNeg,sep="/")

a=read.table(filePos,header=FALSE,sep="\t")
b=read.table(fileNeg,header=FALSE,sep="\t")
pos_gr <- GRanges(
  seqnames = a[,1],
  IRanges(start = as.numeric(a[,2]), end = as.numeric(a[,3]))
)

neg_gr <- GRanges(
    seqnames = b[,1],
  IRanges(start = as.numeric(b[,2]), end = as.numeric(b[,3]))
)
print("bonjovy")
merged_pos_gr <- reduce(pos_gr)
merged_neg_gr <- reduce(neg_gr)

overlap <- findOverlaps(pos_gr, neg_gr)
span=width(intersect(pos_gr,neg_gr))
center_position <- start(intersect(pos_gr,neg_gr)) + width(intersect(pos_gr,neg_gr))/2
matrix=cbind(as.vector(seqnames(intersect(pos_gr,neg_gr))),center_position,center_position)
matrix=matrix[which(span>=100),]
write.table(matrix,paste("/sharedFolder/Data/",i,"_site_CTC.bed",sep=""),quote=FALSE,col.names=FALSE,row.names=FALSE,sep="\t")
}

[1] "bonjovy"
[1] "bonjovy"
[1] "bonjovy"
[1] "bonjovy"


In [29]:
print("bonjovy")



all_data <- data.frame()
condition=c("DMSO","Taze","Vale")
for(junctions_file in list.files("/sharedFolder/Data/1_HTGTS/3_JunctionPaper/junction_pooled/")[-c(1,2,3)])#[c(4,5,6)])
{
    if(length(grep("DMSO",junctions_file))!=0){tss_file <- "/sharedFolder/Data/DMSO_site_CTC.bed"
}
        if(length(grep("Taze",junctions_file))!=0){tss_file <- "/sharedFolder/Data/Taze_site_CTC.bed"
}
    if(length(grep("Vale",junctions_file))!=0){tss_file <- "/sharedFolder/Data/Vale_site_CTC.bed"
}
tss_file <- "/sharedFolder/Data/DMSO_site_CTC.bed"
        tss_file2=read.table(tss_file,header=FALSE,sep="\t")
tss_file <- "/sharedFolder/Data/Taze_site_CTC.bed"
        tss_file2=rbind(tss_file2,read.table(tss_file,header=FALSE,sep="\t"))
tss_file <- "/sharedFolder/Data/Vale_site_CTC.bed"
        tss_file2=rbind(tss_file2,read.table(tss_file,header=FALSE,sep="\t"))
tss_file=tss_file2

colnames(tss_file)=c("chrom", "start", "end")
tss_gr <- GRanges(seqnames = tss_file$chrom, IRanges(start = tss_file$start, end = tss_file$end))
tss_gr=reduce(tss_gr)
    

junctions_file=paste("/sharedFolder/Data/1_HTGTS/3_JunctionPaper/junction_pooled/",junctions_file,sep="")

junctions=read.table(junctions_file,header=FALSE,sep="\t")
colnames(junctions)=c("chrom", "start", "end", "junction", "score", "strand")
junctions_gr <- GRanges(seqnames = junctions$chrom, IRanges(start = junctions$start, end = junctions$end))

#REMOVE MYC and IGH LOCUS 
myc=GRanges(seqnames="chr15",ranges=IRanges(start=60562682,end=63076130))
toRemove=findOverlaps(myc,junctions_gr)@to
igh=GRanges(seqnames="chr12",ranges=IRanges(start=114419999,end=114680000))
toRemove=append(toRemove,findOverlaps(igh,junctions_gr)@to)
junctions_gr=junctions_gr[-toRemove,]
#############################


distance_GR=distanceToNearest(junctions_gr,tss_gr)
distance=start(junctions_gr[queryHits(distance_GR)])- start(tss_gr[subjectHits(distance_GR)])


rpm=log2((countOverlaps(junctions_gr[queryHits(distance_GR)],promoters(junctions_gr,upstream=1000,downstream=1000))+1))
    james=rpm
f=smooth.spline(distance,rpm,spar=0.5)
############################


cycle_data <- data.frame(distance = f$x, rpm = f$y, cycle = fNumber(bnoExt(basename(junctions_file))))#
all_data <- rbind(all_data, cycle_data)

}
all_data=as.data.frame(all_data)

# Crea una colonna separata per gli aloni
highlighted_cycles <- c("AID_KO_Vale", "AID_KO_Taze")
aura_data <- subset(all_data, cycle %in% highlighted_cycles)
aura_data$aura_cycle <- aura_data$cycle

# Colori specifici per l’aura (chiari e distinguibili)
aura_colors <- c(
  "AID_KO_Vale_aura" = "grey",  # blu
  "AID_KO_Taze_aura" = "lightblue"   # #377EB8
)


# Unione tra mapping delle linee e mapping degli aloni
# Assicurati che aura_cycle sia un fattore (questo è il trucco!)
aura_data$aura_cycle <- paste0(aura_data$cycle, "_aura")

plot <- ggplot() +
  geom_line(
    data = aura_data,
    aes(x = distance, y = rpm, color = aura_cycle, group = aura_cycle),
    size = 2.5, alpha = 0.4, show.legend = FALSE
  ) +
  geom_line(
    data = all_data,
    aes(x = distance, y = rpm, color = cycle, linetype = cycle),
    size = 0.9
  ) +
scale_color_manual(values = c(color_mapping, aura_colors)) +
  scale_linetype_manual(values = linetype_mapping) +
  labs(x = "Distance from TSS", y = "Spline fitted junctions distribution",
       title = "Spline fitted junctions distribution vs TSS distance") +
  theme_minimal() +
  theme(legend.position = "top", panel.grid = element_blank()) +
  coord_cartesian(xlim = c(-10000, 10000))







ggsave("/sharedFolder/ConvergentTranscript_WT.pdf", plot, bg = "white")#coord_cartesian(xlim = c(-8000,8000),ylim=c(0,2.2))

[1] "bonjovy"


Warning message in data.frame(distance = f$x, rpm = f$y, cycle = fNumber(bnoExt(basename(junctions_file)))):
“row names were found from a short variable and have been discarded”
Warning message in data.frame(distance = f$x, rpm = f$y, cycle = fNumber(bnoExt(basename(junctions_file)))):
“row names were found from a short variable and have been discarded”
Warning message in data.frame(distance = f$x, rpm = f$y, cycle = fNumber(bnoExt(basename(junctions_file)))):
“row names were found from a short variable and have been discarded”
Warning message in data.frame(distance = f$x, rpm = f$y, cycle = fNumber(bnoExt(basename(junctions_file)))):
“row names were found from a short variable and have been discarded”
Warning message in data.frame(distance = f$x, rpm = f$y, cycle = fNumber(bnoExt(basename(junctions_file)))):
“row names were found from a short variable and have been discarded”
Warning message in data.frame(distance = f$x, rpm = f$y, cycle = fNumber(bnoExt(basename(junctions_file)))):
“

In [30]:
print("bonjovy")



all_data <- data.frame()
condition=c("DMSO","Taze","Vale")
for(junctions_file in list.files("/sharedFolder/Data/1_HTGTS/3_JunctionPaper/junction_pooled/")[c(1,2,3,7,8,9)])#[c(4,5,6)])
{
    if(length(grep("DMSO",junctions_file))!=0){tss_file <- "/sharedFolder/Data/DMSO_site_CTC.bed"
}
        if(length(grep("Taze",junctions_file))!=0){tss_file <- "/sharedFolder/Data/Taze_site_CTC.bed"
}
    if(length(grep("Vale",junctions_file))!=0){tss_file <- "/sharedFolder/Data/Vale_site_CTC.bed"
}
tss_file <- "/sharedFolder/Data/DMSO_site_CTC.bed"
        tss_file2=read.table(tss_file,header=FALSE,sep="\t")
tss_file <- "/sharedFolder/Data/Taze_site_CTC.bed"
        tss_file2=rbind(tss_file2,read.table(tss_file,header=FALSE,sep="\t"))
tss_file <- "/sharedFolder/Data/Vale_site_CTC.bed"
        tss_file2=rbind(tss_file2,read.table(tss_file,header=FALSE,sep="\t"))
tss_file=tss_file2

colnames(tss_file)=c("chrom", "start", "end")
tss_gr <- GRanges(seqnames = tss_file$chrom, IRanges(start = tss_file$start, end = tss_file$end))
tss_gr=reduce(tss_gr)
    

junctions_file=paste("/sharedFolder/Data/1_HTGTS/3_JunctionPaper/junction_pooled/",junctions_file,sep="")

junctions=read.table(junctions_file,header=FALSE,sep="\t")
colnames(junctions)=c("chrom", "start", "end", "junction", "score", "strand")
junctions_gr <- GRanges(seqnames = junctions$chrom, IRanges(start = junctions$start, end = junctions$end))

#REMOVE MYC and IGH LOCUS 
myc=GRanges(seqnames="chr15",ranges=IRanges(start=60562682,end=63076130))
toRemove=findOverlaps(myc,junctions_gr)@to
igh=GRanges(seqnames="chr12",ranges=IRanges(start=114419999,end=114680000))
toRemove=append(toRemove,findOverlaps(igh,junctions_gr)@to)
junctions_gr=junctions_gr[-toRemove,]
#############################


distance_GR=distanceToNearest(junctions_gr,tss_gr)
distance=start(junctions_gr[queryHits(distance_GR)])- start(tss_gr[subjectHits(distance_GR)])


rpm=log2((countOverlaps(junctions_gr[queryHits(distance_GR)],promoters(junctions_gr,upstream=1000,downstream=1000))+1))
    james=rpm
f=smooth.spline(distance,rpm,spar=0.5)
############################


cycle_data <- data.frame(distance = f$x, rpm = f$y, cycle = fNumber(bnoExt(basename(junctions_file))))#
all_data <- rbind(all_data, cycle_data)

}
all_data=as.data.frame(all_data)




# Crea una colonna separata per gli aloni
highlighted_cycles <- c("AID_KO_Vale", "AID_KO_Taze")
aura_data <- subset(all_data, cycle %in% highlighted_cycles)
aura_data$aura_cycle <- aura_data$cycle

# Colori specifici per l’aura (chiari e distinguibili)
aura_colors <- c(
  "AID_KO_Vale_aura" = "grey",  # blu
  "AID_KO_Taze_aura" = "lightblue"   # #377EB8
)


# Unione tra mapping delle linee e mapping degli aloni
# Assicurati che aura_cycle sia un fattore (questo è il trucco!)
aura_data$aura_cycle <- paste0(aura_data$cycle, "_aura")

plot <- ggplot() +
  geom_line(
    data = aura_data,
    aes(x = distance, y = rpm, color = aura_cycle, group = aura_cycle),
    size = 2.5, alpha = 0.4, show.legend = FALSE
  ) +
  geom_line(
    data = all_data,
    aes(x = distance, y = rpm, color = cycle, linetype = cycle),
    size = 0.9
  ) +
scale_color_manual(values = c(color_mapping, aura_colors)) +
  scale_linetype_manual(values = linetype_mapping) +
  labs(x = "Distance from TSS", y = "Spline fitted junctions distribution",
       title = "Spline fitted junctions distribution vs TSS distance") +
  theme_minimal() +
  theme(legend.position = "top", panel.grid = element_blank()) +
  coord_cartesian(xlim = c(-10000, 10000))


ggsave("/sharedFolder/ConvergentTranscript_Ligase.pdf", plot, bg = "white")#coord_cartesian(xlim = c(-8000,8000),ylim=c(0,2.2))


[1] "bonjovy"


Warning message in data.frame(distance = f$x, rpm = f$y, cycle = fNumber(bnoExt(basename(junctions_file)))):
“row names were found from a short variable and have been discarded”
Warning message in data.frame(distance = f$x, rpm = f$y, cycle = fNumber(bnoExt(basename(junctions_file)))):
“row names were found from a short variable and have been discarded”
Warning message in data.frame(distance = f$x, rpm = f$y, cycle = fNumber(bnoExt(basename(junctions_file)))):
“row names were found from a short variable and have been discarded”
Warning message in data.frame(distance = f$x, rpm = f$y, cycle = fNumber(bnoExt(basename(junctions_file)))):
“row names were found from a short variable and have been discarded”
Warning message in data.frame(distance = f$x, rpm = f$y, cycle = fNumber(bnoExt(basename(junctions_file)))):
“row names were found from a short variable and have been discarded”
Warning message in data.frame(distance = f$x, rpm = f$y, cycle = fNumber(bnoExt(basename(junctions_file)))):
“

In [34]:
print("bonjovy")



all_data <- data.frame()
condition=c("DMSO","Taze","Vale")
for(junctions_file in list.files("/sharedFolder/Data/1_HTGTS/3_JunctionPaper/junction_pooled/")[-c(1,2,3)])#[c(4,5,6)])
{
    if(length(grep("DMSO",junctions_file))!=0){tss_file <- "/sharedFolder/Data/5_CutTagAce/MACS2_220607_JT10123_MN00260_trimmed_DMSO/macs2_peak_q0.1_peaks.csv"
}
        if(length(grep("Taze",junctions_file))!=0){tss_file <- "/sharedFolder/Data/5_CutTagAce/MACS2_220607_JT10123_MN00260_trimmed_Taze/macs2_peak_q0.1_peaks.csv"
}
   if(length(grep("Vale",junctions_file))!=0){tss_file <- "/sharedFolder/Data/5_CutTagAce/MACS2_220607_JT10123_MN00260_trimmed_Vale/macs2_peak_q0.1_peaks.csv"
}
tss_file <- "/sharedFolder/Data/5_CutTagAce/MACS2_220607_JT10123_MN00260_trimmed_DMSO/macs2_peak_q0.1_peaks.csv"
tss_file2=read.table(tss_file,header=TRUE,sep=",")[,c(1,2,3)]
tss_file <- "/sharedFolder/Data/5_CutTagAce/MACS2_220607_JT10123_MN00260_trimmed_Taze/macs2_peak_q0.1_peaks.csv"
tss_file2=rbind(tss_file2,read.table(tss_file,header=TRUE,sep=",")[,c(1,2,3)])
tss_file <- "/sharedFolder/Data/5_CutTagAce/MACS2_220607_JT10123_MN00260_trimmed_Vale/macs2_peak_q0.1_peaks.csv"
tss_file2=rbind(tss_file2,read.table(tss_file,header=TRUE,sep=",")[,c(1,2,3)])
tss_file=tss_file2

    
#tss_file=read.table(tss_file,header=TRUE,sep=",")[,c(1,2,3)]
colnames(tss_file)=c("chrom", "start", "end")
tss_file_center=(tss_file$start + tss_file$end) / 2
tss_file$start=tss_file_center
tss_file$end=tss_file_center
tss_gr <- GRanges(seqnames = tss_file$chrom, IRanges(start = tss_file$start, end = tss_file$end))
    
tss_gr=reduce(tss_gr)
junctions_file=paste("/sharedFolder/Data/1_HTGTS/3_JunctionPaper/junction_pooled/",junctions_file,sep="")

junctions=read.table(junctions_file,header=FALSE,sep="\t")
colnames(junctions)=c("chrom", "start", "end", "junction", "score", "strand")
junctions_gr <- GRanges(seqnames = junctions$chrom, IRanges(start = junctions$start, end = junctions$end))

#REMOVE MYC and IGH LOCUS 
myc=GRanges(seqnames="chr15",ranges=IRanges(start=60562682,end=63076130))
toRemove=findOverlaps(myc,junctions_gr)@to
igh=GRanges(seqnames="chr12",ranges=IRanges(start=114419999,end=114680000))
toRemove=append(toRemove,findOverlaps(igh,junctions_gr)@to)
junctions_gr=junctions_gr[-toRemove,]
#############################


distance_GR=distanceToNearest(junctions_gr,tss_gr)
distance=start(junctions_gr[queryHits(distance_GR)])- start(tss_gr[subjectHits(distance_GR)])


rpm=log2(countOverlaps(junctions_gr[queryHits(distance_GR)],promoters(junctions_gr,upstream=5000,downstream=5000))+1)
#rpm=countOverlaps(junctions_gr[queryHits(distance_GR)],promoters(junctions_gr,upstream=5000,downstream=5000))

f=smooth.spline(distance,rpm,spar=1)
############################


cycle_data <- data.frame(distance = f$x, rpm = f$y, cycle = fNumber(bnoExt(basename(junctions_file))))#
all_data <- rbind(all_data, cycle_data)

}
all_data=as.data.frame(all_data)
# Crea una colonna separata per gli aloni
highlighted_cycles <- c("AID_KO_Vale", "AID_KO_Taze")
aura_data <- subset(all_data, cycle %in% highlighted_cycles)
aura_data$aura_cycle <- aura_data$cycle

# Colori specifici per l’aura (chiari e distinguibili)
aura_colors <- c(
  "AID_KO_Vale_aura" = "grey",  # blu
  "AID_KO_Taze_aura" = "lightblue"   # #377EB8
)


# Unione tra mapping delle linee e mapping degli aloni
# Assicurati che aura_cycle sia un fattore (questo è il trucco!)
aura_data$aura_cycle <- paste0(aura_data$cycle, "_aura")

plot <- ggplot() +
  geom_line(
    data = aura_data,
    aes(x = distance, y = rpm, color = aura_cycle, group = aura_cycle),
    size = 2.5, alpha = 0.4, show.legend = FALSE
  ) +
  geom_line(
    data = all_data,
    aes(x = distance, y = rpm, color = cycle, linetype = cycle),
    size = 0.9
  ) +
scale_color_manual(values = c(color_mapping, aura_colors)) +
  scale_linetype_manual(values = linetype_mapping) +
  labs(x = "Distance from TSS", y = "Spline fitted junctions distribution",
       title = "Spline fitted junctions distribution vs TSS distance") +
  theme_minimal() +
  theme(legend.position = "top", panel.grid = element_blank()) +
  coord_cartesian(xlim = c(-100000, 100000))
ggsave("/sharedFolder/SuperEnhancer_WT.pdf", plot, bg = "white")#coord_cartesian(xlim = c(-8000,8000),ylim=c(0,2.2))

[1] "bonjovy"


Warning message in data.frame(distance = f$x, rpm = f$y, cycle = fNumber(bnoExt(basename(junctions_file)))):
“row names were found from a short variable and have been discarded”
Warning message in .Seqinfo.mergexy(x, y):
“Each of the 2 combined objects has sequence levels not in the other:
  - in 'x': chrY
  - in 'y': chr13_random, chr17_random, chr1_random, chr4_random, chr8_random, chr9_random, chrUn_random, chrY_random, chr3_random, chrX_random
  Make sure to always combine/compare objects based on the same reference
  genome (use suppressWarnings() to suppress this warning).”
Warning message in data.frame(distance = f$x, rpm = f$y, cycle = fNumber(bnoExt(basename(junctions_file)))):
“row names were found from a short variable and have been discarded”
Warning message in data.frame(distance = f$x, rpm = f$y, cycle = fNumber(bnoExt(basename(junctions_file)))):
“row names were found from a short variable and have been discarded”
Warning message in data.frame(distance = f$x, rpm = f$y, 

In [33]:
print("bonjovy")



all_data <- data.frame()
condition=c("DMSO","Taze","Vale")
for(junctions_file in list.files("/sharedFolder/Data/1_HTGTS/3_JunctionPaper/junction_pooled/")[c(1,2,3,7,8,9)])#[c(4,5,6)])
{
    if(length(grep("DMSO",junctions_file))!=0){tss_file <- "/sharedFolder/Data/5_CutTagAce/MACS2_220607_JT10123_MN00260_trimmed_DMSO/macs2_peak_q0.1_peaks.csv"
}
        if(length(grep("Taze",junctions_file))!=0){tss_file <- "/sharedFolder/Data/5_CutTagAce/MACS2_220607_JT10123_MN00260_trimmed_Taze/macs2_peak_q0.1_peaks.csv"
}
   if(length(grep("Vale",junctions_file))!=0){tss_file <- "/sharedFolder/Data/5_CutTagAce/MACS2_220607_JT10123_MN00260_trimmed_Vale/macs2_peak_q0.1_peaks.csv"
}
tss_file <- "/sharedFolder/Data/5_CutTagAce/MACS2_220607_JT10123_MN00260_trimmed_DMSO/macs2_peak_q0.1_peaks.csv"
tss_file2=read.table(tss_file,header=TRUE,sep=",")[,c(1,2,3)]
tss_file <- "/sharedFolder/Data/5_CutTagAce/MACS2_220607_JT10123_MN00260_trimmed_Taze/macs2_peak_q0.1_peaks.csv"
tss_file2=rbind(tss_file2,read.table(tss_file,header=TRUE,sep=",")[,c(1,2,3)])
tss_file <- "/sharedFolder/Data/5_CutTagAce/MACS2_220607_JT10123_MN00260_trimmed_Vale/macs2_peak_q0.1_peaks.csv"
tss_file2=rbind(tss_file2,read.table(tss_file,header=TRUE,sep=",")[,c(1,2,3)])
tss_file=tss_file2

    
#tss_file=read.table(tss_file,header=TRUE,sep=",")[,c(1,2,3)]
colnames(tss_file)=c("chrom", "start", "end")
tss_file_center=(tss_file$start + tss_file$end) / 2
tss_file$start=tss_file_center
tss_file$end=tss_file_center
tss_gr <- GRanges(seqnames = tss_file$chrom, IRanges(start = tss_file$start, end = tss_file$end))
    
tss_gr=reduce(tss_gr)
junctions_file=paste("/sharedFolder/Data/1_HTGTS/3_JunctionPaper/junction_pooled/",junctions_file,sep="")

junctions=read.table(junctions_file,header=FALSE,sep="\t")
colnames(junctions)=c("chrom", "start", "end", "junction", "score", "strand")
junctions_gr <- GRanges(seqnames = junctions$chrom, IRanges(start = junctions$start, end = junctions$end))

#REMOVE MYC and IGH LOCUS 
myc=GRanges(seqnames="chr15",ranges=IRanges(start=60562682,end=63076130))
toRemove=findOverlaps(myc,junctions_gr)@to
igh=GRanges(seqnames="chr12",ranges=IRanges(start=114419999,end=114680000))
toRemove=append(toRemove,findOverlaps(igh,junctions_gr)@to)
junctions_gr=junctions_gr[-toRemove,]
#############################


distance_GR=distanceToNearest(junctions_gr,tss_gr)
distance=start(junctions_gr[queryHits(distance_GR)])- start(tss_gr[subjectHits(distance_GR)])


rpm=log2(countOverlaps(junctions_gr[queryHits(distance_GR)],promoters(junctions_gr,upstream=5000,downstream=5000))+1)
f=smooth.spline(distance,rpm,spar=0.99)
############################


cycle_data <- data.frame(distance = f$x, rpm = f$y, cycle = fNumber(bnoExt(basename(junctions_file))))#
all_data <- rbind(all_data, cycle_data)

}
all_data=as.data.frame(all_data)
if(length(which(all_data$rpm<=0))!=0){
all_data=all_data[-which(all_data$rpm<=0),]}
print("finito")

# Crea una colonna separata per gli aloni
highlighted_cycles <- c("AID_KO_Vale", "AID_KO_Taze")
aura_data <- subset(all_data, cycle %in% highlighted_cycles)
aura_data$aura_cycle <- aura_data$cycle

# Colori specifici per l’aura (chiari e distinguibili)
aura_colors <- c(
  "AID_KO_Vale_aura" = "grey",  # blu
  "AID_KO_Taze_aura" = "lightblue"   # #377EB8
)


# Unione tra mapping delle linee e mapping degli aloni
# Assicurati che aura_cycle sia un fattore (questo è il trucco!)
aura_data$aura_cycle <- paste0(aura_data$cycle, "_aura")

plot <- ggplot() +
  geom_line(
    data = aura_data,
    aes(x = distance, y = rpm, color = aura_cycle, group = aura_cycle),
    size = 2.5, alpha = 0.4, show.legend = FALSE
  ) +
  geom_line(
    data = all_data,
    aes(x = distance, y = rpm, color = cycle, linetype = cycle),
    size = 0.9
  ) +
scale_color_manual(values = c(color_mapping, aura_colors)) +
  scale_linetype_manual(values = linetype_mapping) +
  labs(x = "Distance from TSS", y = "Spline fitted junctions distribution",
       title = "Spline fitted junctions distribution vs TSS distance") +
  theme_minimal() +
  theme(legend.position = "top", panel.grid = element_blank()) +
  coord_cartesian(xlim = c(-100000, 100000))
ggsave("/sharedFolder/superEnhancer_Ligase.pdf", plot, bg = "white")#coord_cartesian(xlim = c(-8000,8000),ylim=c(0,2.2))

[1] "bonjovy"


Warning message in .Seqinfo.mergexy(x, y):
“Each of the 2 combined objects has sequence levels not in the other:
  - in 'x': chrY
  - in 'y': chr13_random, chr17_random, chr1_random, chr4_random, chr8_random, chr9_random, chrUn_random, chrY_random, chr3_random, chrX_random
  Make sure to always combine/compare objects based on the same reference
  genome (use suppressWarnings() to suppress this warning).”
Warning message in data.frame(distance = f$x, rpm = f$y, cycle = fNumber(bnoExt(basename(junctions_file)))):
“row names were found from a short variable and have been discarded”
Warning message in .Seqinfo.mergexy(x, y):
“Each of the 2 combined objects has sequence levels not in the other:
  - in 'x': chrY
  - in 'y': chr13_random, chr17_random, chr1_random, chr4_random, chr8_random, chr9_random, chrUn_random, chrY_random, chr3_random, chrX_random
  Make sure to always combine/compare objects based on the same reference
  genome (use suppressWarnings() to suppress this warning).”
Warn

[1] "finito"


Saving 6.67 x 6.67 in image
